In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv('gutenberg_books.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Author                 4960 non-null   object
 1   Title                  4960 non-null   object
 2   Note                   4960 non-null   object
 3   Summary                4960 non-null   object
 4   Language               4960 non-null   object
 5   LoC Classes            4957 non-null   object
 6   Subjects               4956 non-null   object
 7   Category               4960 non-null   object
 8   EBook-No.              4960 non-null   int64 
 9   Release Date           4960 non-null   object
 10  Most Recently Updated  4960 non-null   object
 11  Copyright Status       4960 non-null   object
dtypes: int64(1), object(11)
memory usage: 465.1+ KB


In [3]:
import pandas as pd
from langchain.memory import ConversationBufferMemory
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
import os 
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
print(os.getenv('OPENAI_API_KEY'))

None


In [6]:
azure_openai_api_key = os.getenv("OPENAI_API_KEYAZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

In [7]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import AzureOpenAI

In [8]:
llm = AzureChatOpenAI(api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_api_endpoint,
                        model=deployment_name,
                        temperature=0.9
                        )


C:\Users\valen\AppData\Local\Temp\ipykernel_1388\3775370592.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(api_key=azure_openai_api_key,


In [9]:
file = './gutenberg_books.csv'
loader = CSVLoader(file_path=file, encoding='ISO-8859-1')

In [10]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter

In [11]:
from langchain_core.vectorstores import InMemoryVectorStore

embedding_model = AzureOpenAIEmbeddings(openai_api_key=azure_openai_api_key,
                                    azure_deployment='text-embedding-3-large',
                                    azure_endpoint=azure_openai_api_endpoint,
                                    openai_api_version="2023-05-15",
                                    chunk_size=500
)

index = VectorstoreIndexCreator(
    embedding = embedding_model,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

C:\Users\valen\AppData\Local\Temp\ipykernel_1388\2842684352.py:3: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(openai_api_key=azure_openai_api_key,
c:\Users\valen\formation\Mon_rpo\Gutenberg\env\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [12]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
vectorstore = FAISS.from_documents(texts, embedding_model)

In [13]:
retriever = vectorstore.as_retriever()

In [14]:
vectorstore.save_local('gutenberg_books_vectorstore')

In [15]:
faiss_index = FAISS.load_local('gutenberg_books_vectorstore', embedding_model, allow_dangerous_deserialization=True)

In [23]:
query = "what the summary of the bible ?"
docs_with_similar_embeddings = faiss_index.similarity_search(query, k=1)


In [24]:
for doc in docs_with_similar_embeddings:
    print(doc.page_content)

Author: Unknown
Title: The Bible, King James Version, Complete
Note: No Note
Summary: No Summary
Language: Unknown
LoC Classes: BS: Philosophy, Psychology, Religion: Christianity: The Bible, Old and New Testament
Subjects: Bible
Category: No Category
EBook-No.: 30
Release Date: No Release Date
Most Recently Updated: No Update Date
Copyright Status: Unknown
